In [1]:
import pandas as pd
import numpy as np

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
from numpy import array

## importing libraries

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import time
from sklearn import preprocessing
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.metrics
import math
from numpy import mean
from numpy import std

from sklearn.metrics import mean_squared_error, r2_score


print('Done importing libraries')

Done importing libraries


In [2]:
df2 = pd.read_csv('Data/allFall_delhi.csv')
df2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df2.drop(["index"], axis=1, inplace=True)



Ozone = df2['O3']

OzoneP24 = Ozone.shift(-24)
OzoneP24 = OzoneP24.replace(np.nan, 54.94)
df2['O3P24'] = OzoneP24

df2.info()
print(df2.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7320 entries, 0 to 7319
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   a            7320 non-null   int64  
 1   City         7320 non-null   object 
 2   Date         7320 non-null   object 
 3   Time         7320 non-null   object 
 4   PM2.5        7320 non-null   float64
 5   PM10         7320 non-null   float64
 6   NO           7320 non-null   float64
 7   NO2          7320 non-null   float64
 8   NOx          7320 non-null   float64
 9   NH3          7320 non-null   float64
 10  CO           7320 non-null   float64
 11  SO2          7320 non-null   float64
 12  O3           7320 non-null   float64
 13  Benzene      7320 non-null   float64
 14  Toluene      7320 non-null   float64
 15  Xylene       7320 non-null   float64
 16  O3P24        7320 non-null   float64
 17  Temperature  7320 non-null   float64
dtypes: float64(14), int64(1), object(3)
memory usage

In [10]:
## defining variables

#X = df2[['PM10', 'NO', 'NO2', 'CO', 'SO2', 'O3', 'Toluene', 'Temp']] # , 'Humid', Xylene
X = df2[['PM10', 'NO', 'NO2', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train1.shape[0]))
print("Size of testing dataset: {} rows".format(X_test1.shape[0]))

print('Done defining variables')

Size of training dataset: 6588 rows
Size of testing dataset: 732 rows
Done defining variables


In [11]:
## scaling variables

'''importing libraries'''
from sklearn.preprocessing import RobustScaler

scaler = preprocessing.RobustScaler()
X_train_scaled1 = scaler.fit_transform(X_train1)
X_test_scaled1 = scaler.fit_transform(X_test1)

print('Done scaling :)')

Done scaling :)


In [14]:
## linear regression
t0 = time.time()

'''importing library'''
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold

'''create linear regressor object'''
lnRegressor = LinearRegression()

'''fitting regressor'''
lnRegressor.fit(X_train1, y_train)

# cv stuff

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

cvln_r2scores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvln_r2scores)
cvln_rmsescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvln_rmsescores)
cvln_maescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvln_maescores)

# calculating r^2 adj score

cvln_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvln_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvln_adj.append(adj_r2)    

cvln_r2score = round((np.mean(cvln_r2scores)), 4)
cvln_adjscore = round((np.mean(cvln_adj)), 4)
cvln_rmsescore = (round(abs(np.mean(cvln_rmsescores)),2))
cvln_maescore = (round(abs(np.mean(cvln_maescores)),2))


print(cvln_r2score, cvln_adjscore, cvln_rmsescore, cvln_maescore)

t1 = time.time()
total = t1-t0

print("done training linear regressor | time taken: %f seconds" %total)
yPredln = lnRegressor.predict(X_test1)
print('The average R2 value for linear regressor is :', cvln_r2score)

[0.32026539 0.28527367 0.49956145 0.46885147 0.33368824 0.40061604
 0.47884701 0.3490707  0.33856104 0.37221278]
[-33.13575608 -36.49598079 -30.83252573 -29.75087474 -30.99669513
 -31.24351898 -31.77566834 -37.07519601 -34.25850498 -32.45071191]
[-20.80310253 -21.22417279 -20.28516262 -20.41383187 -20.99601429
 -19.83259719 -19.74967884 -21.87471065 -20.95413234 -19.83221266]
0.3847 0.3838 32.8 20.6
done training linear regressor | time taken: 0.392385 seconds
The average R2 value for linear regressor is : 0.3847


In [15]:
## kneighbors regression
print("STARTING: KN")

t0 = time.time()

'''importing library'''
from sklearn.neighbors import KNeighborsRegressor

'''create regressor object'''
knRegressor = KNeighborsRegressor(n_neighbors = 4, metric = 'minkowski', p = 1)

'''fitting regressor'''
knRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvkn_r2scores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvkn_r2scores)
cvkn_rmsescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvkn_rmsescores)
cvkn_maescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvkn_maescores)

# calculating r^2 adj score

cvkn_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvkn_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvkn_adj.append(adj_r2)    

cvkn_r2score = round((np.mean(cvkn_r2scores)), 4)
cvkn_adjscore = round((np.mean(cvkn_adj)), 4)
cvkn_rmsescore = (round(abs(np.mean(cvkn_rmsescores)),2))
cvkn_maescore = (round(abs(np.mean(cvkn_maescores)),2))

print(cvkn_r2score, cvkn_adjscore, cvkn_rmsescore, cvkn_maescore)


t1 = time.time()
total = t1-t0

print("done training knn regressor | time taken: %f seconds" %total)

yPredkn = knRegressor.predict(X_test_scaled1)
print('The average R2 value for KNN Regressor is :', cvln_r2score)

STARTING: KN
[0.46407564 0.62066274 0.65991734 0.55472344 0.39121916 0.67157376
 0.58331671 0.67151124 0.64444122 0.6278565 ]
[-29.42245738 -26.58813661 -25.41709841 -27.23997229 -29.62832773
 -23.12737304 -28.412871   -26.33763671 -25.11766279 -24.98464672]
[-16.80531013 -16.19481411 -14.93703208 -16.3475852  -16.56856539
 -14.77811898 -15.59477114 -15.8037747  -15.53974301 -14.42270281]
0.5889 0.5883 26.63 15.7
done training knn regressor | time taken: 2.019506 seconds
The average R2 value for KNN Regressor is : 0.3847


In [16]:
## svr regressor
print("STARTING: SVR")

t0 = time.time()

'''importing library'''
from sklearn.svm import SVR

'''create regressor object'''
svrRegressor = SVR(C = 10, gamma = 0.1, kernel= 'rbf')

'''fitting regressor'''
svrRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvsvr_r2scores = cross_val_score(svrRegressor, X_train_scaled1, y_train, cv = 10, scoring = 'r2')
print(cvsvr_r2scores)
cvsvr_rmsescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvsvr_rmsescores)
cvsvr_maescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvsvr_maescores)


# calculating r^2 adj score

cvsvr_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvsvr_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvsvr_adj.append(adj_r2)

cvsvr_r2score = round((np.mean(cvsvr_r2scores)), 4)
cvsvr_adjscore = round((np.mean(cvsvr_adj)), 4)
cvsvr_rmsescore = (round(abs(np.mean(cvsvr_rmsescores)),2))
cvsvr_maescore = (round(abs(np.mean(cvsvr_maescores)),2))

print(cvsvr_r2score, cvsvr_adjscore, cvsvr_rmsescore, cvsvr_maescore)

t1 = time.time()
total = t1-t0

print("done training svr regressor | time taken: %f seconds" %total)

yPredsvr = svrRegressor.predict(X_test_scaled1)
print('The average R2 value for SVR Regressor is :', cvsvr_r2score)

STARTING: SVR
[0.42431359 0.37172612 0.49439835 0.45765963 0.51091743 0.43789555
 0.41525857 0.41274261 0.42916908 0.47301932]
[-41.2117769  -44.06144826 -45.44846251 -41.52536143 -39.1982607
 -41.47946456 -45.60051466 -47.38909043 -42.9924516  -41.80088174]
[-29.32309723 -29.88109571 -33.16947386 -30.95463449 -30.50863809
 -30.13864559 -31.60834233 -32.00647415 -30.32783202 -30.50880739]
0.4427 0.4419 43.07 30.84
done training svr regressor | time taken: 262.592602 seconds
The average R2 value for SVR Regressor is : 0.4427


In [17]:
## defining variables WITH MULTICOLLINEARITY: random forest, decision tree, maybe adaboost, and xgboost

X = df2[['PM10', 'NO', 'NO2', 'NOx', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train.shape[0]))
print("Size of testing dataset: {} rows".format(X_test.shape[0]))

print('Done defining NEW variables')

Size of training dataset: 6588 rows
Size of testing dataset: 732 rows
Done defining NEW variables


In [18]:
## random forest regressor
print("STARTING: rf")
t0 = time.time()

'''importing library'''
from sklearn.ensemble import RandomForestRegressor

'''create regressor object'''
rfRegressor = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=250) 

'''fitting regressor'''
rfRegressor.fit(X_train, y_train)

# cv stuff

cvrf_r2scores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'r2') # x_train was scaled for some reason, check if it changes things?
print(cvrf_r2scores)
cvrf_rmsescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvrf_rmsescores)
cvrf_maescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvrf_maescores)


# calculating r^2 adj score

cvrf_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvrf_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvrf_adj.append(adj_r2)

cvrf_r2score = round((np.mean(cvrf_r2scores)), 4)
cvrf_adjscore = round((np.mean(cvrf_adj)), 4)
cvrf_rmsescore = (round(abs(np.mean(cvrf_rmsescores)),2))
cvrf_maescore = (round(abs(np.mean(cvrf_maescores)),2))

print(cvrf_r2score, cvrf_adjscore, cvrf_rmsescore, cvrf_maescore)

print(cvrf_r2score, cvrf_adjscore)

t1 = time.time()
total = t1-t0

print("done training random forest regressor | time taken: %f seconds" %total)
yPredrf = rfRegressor.predict(X_test)
print('The average R2 value for Random Forest Regressor is :', cvrf_r2score)

## printing feature importances

importances = rfRegressor.feature_importances_

sorted_indices = np.argsort(importances)[::-1]

print('feature importances:::')
print(sorted_indices)

STARTING: rf
[0.64832703 0.59362337 0.7117066  0.62370947 0.57214386 0.69173111
 0.59662629 0.6339325  0.66129221 0.67005621]
[-23.83398648 -27.51943475 -23.4018976  -25.0410992  -24.83850981
 -22.40640761 -27.95541033 -27.80335025 -24.51523794 -23.52545195]
[-14.94503521 -15.70992004 -14.63032132 -15.06532281 -15.02689965
 -14.3236038  -15.3422457  -15.31698326 -15.04742006 -13.67048764]
0.6403 0.6397 25.08 14.91
0.6403 0.6397
done training random forest regressor | time taken: 1073.418403 seconds
The average R2 value for Random Forest Regressor is : 0.6403
feature importances:::
[ 7  1  3  4  2  8  6  0  5 10  9]


In [19]:
## decision tree regressor
print('STARTING: dt')
t0 = time.time()

'''importing library'''
from sklearn.tree import DecisionTreeRegressor

'''create regressor object'''
dtRegressor = DecisionTreeRegressor(random_state=0, max_depth = 6)

'''fitting regressor'''
dtRegressor.fit(X_train,y_train)

# cv stuff

cvdt_r2scores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvdt_r2scores)
cvdt_rmsescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvdt_rmsescores)
cvdt_maescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvdt_maescores)


# calculating r^2 adj score

cvdt_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvdt_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvdt_adj.append(adj_r2)

cvdt_r2score = round((np.mean(cvdt_r2scores)), 4)
cvdt_adjscore = round((np.mean(cvdt_adj)), 4)
cvdt_rmsescore = (round(abs(np.mean(cvdt_rmsescores)),2))
cvdt_maescore = (round(abs(np.mean(cvdt_maescores)),2))

print(cvdt_r2score, cvdt_adjscore, cvdt_rmsescore, cvdt_maescore)

t1 = time.time()
total = t1-t0

print("done training decision tree regressor | time taken: %f seconds" %total)

yPreddt = dtRegressor.predict(X_test)
print('The average R2 value for Decision Tree Regressor is :', cvdt_r2score)

STARTING: dt
[0.3890984  0.40715918 0.59598231 0.51715482 0.10130807 0.47768201
 0.52179297 0.62493211 0.52275389 0.61343404]
[-31.4132467  -33.23871281 -27.70345357 -28.36584314 -35.99830681
 -29.1658658  -30.43827626 -28.14307106 -29.10011599 -25.46418611]
[-18.27096348 -19.1058252  -17.28815632 -17.33385933 -19.02967799
 -17.85336055 -17.46541274 -17.40754279 -17.57887826 -15.71521455]
0.4771 0.4763 29.9 17.7
done training decision tree regressor | time taken: 2.422012 seconds
The average R2 value for Decision Tree Regressor is : 0.4771


In [20]:
## adaboost regressor
print('STARTING: adaboost regressor')
t0 = time.time()

'''importing library'''
from sklearn.ensemble import AdaBoostRegressor

'''create regressor object'''
adaRegressor = AdaBoostRegressor(random_state=0, learning_rate = 0.1, n_estimators=100)

'''fitting regressor'''
adaRegressor.fit(X_train, y_train)

# cv stuff

cvada_r2scores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvada_r2scores)
cvada_rmsescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvada_rmsescores)
cvada_maescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvada_maescores)

# calculating r^2 adj score

cvada_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvada_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvada_adj.append(adj_r2)

cvada_r2score = round((np.mean(cvada_r2scores)), 4)
cvada_adjscore = round((np.mean(cvada_adj)), 4)
cvada_rmsescore = (round(abs(np.mean(cvada_rmsescores)),2))
cvada_maescore = (round(abs(np.mean(cvada_maescores)),2))

print(cvada_r2score, cvada_adjscore, cvada_rmsescore, cvada_maescore)

t1 = time.time()
total = t1-t0

print("done training Adaboost regressor | time taken: %f seconds" %total)

yPredada = adaRegressor.predict(X_test)
print('The average R2 value for Adaboost Regressor is :', cvada_r2score)

STARTING: adaboost regressor
[0.38352363 0.43645062 0.50216289 0.45578189 0.24155811 0.44554236
 0.5189864  0.41503125 0.43343763 0.47300607]
[-31.55625143 -32.40717226 -30.75228276 -30.11467905 -33.07027519
 -30.0497989  -30.52746577 -35.14656441 -31.70643287 -29.73177246]
[-22.81522008 -23.3371048  -22.56439468 -23.12536314 -23.52035335
 -22.6473323  -20.75158973 -23.6598526  -23.0496313  -22.21528464]
0.4305 0.4296 31.51 22.77
done training Adaboost regressor | time taken: 95.046222 seconds
The average R2 value for Adaboost Regressor is : 0.4305


In [21]:
## xgboost regressor
print('STARTING: xgboost regressor')
t0 = time.time()

'''importing library'''
import xgboost as xgb

'''create regressor object'''
xgbRegressor = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, n_estimators=300, verbosity = 0, random_state = 0, silent = True)

'''fitting regressor'''
xgbRegressor.fit(X_train, y_train)

# cv stuff

cvxgb_r2scores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvxgb_r2scores)
cvxgb_rmsescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvxgb_rmsescores)
cvxgb_maescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvxgb_maescores)


# calculating r^2 adj score

cvxgb_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvxgb_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvxgb_adj.append(adj_r2)

cvxgb_r2score = round((np.mean(cvxgb_r2scores)), 4)
cvxgb_adjscore = round((np.mean(cvxgb_adj)), 4)
cvxgb_rmsescore = (round(abs(np.mean(cvxgb_rmsescores)),2))
cvxgb_maescore = (round(abs(np.mean(cvxgb_maescores)),2))

print(cvxgb_r2score, cvxgb_adjscore, cvxgb_rmsescore, cvxgb_maescore)

t1 = time.time()
total = t1-t0

print("done training xgboost regressor | time taken: %f seconds" %total)

yPredxgb = xgbRegressor.predict(X_test)
print('The average R2 value for xgboost Regressor is :', cvxgb_r2score)

STARTING: xgboost regressor
[0.66385583 0.60605693 0.69213472 0.64184638 0.51992473 0.70113436
 0.61715384 0.55477376 0.68405147 0.67579502]
[-23.30182764 -27.09517007 -24.18321734 -24.43016618 -26.3106345
 -22.06202509 -27.23480218 -30.66245476 -23.67727337 -23.31996183]
[-14.61349986 -15.55168113 -14.60653365 -14.61288832 -14.96284902
 -13.90012721 -14.86793313 -15.74012334 -14.38737653 -13.2580433 ]
0.6357 0.6351 25.23 14.65
done training xgboost regressor | time taken: 472.176263 seconds
The average R2 value for xgboost Regressor is : 0.6357


In [22]:
##### printing cross-validations scores
# printing cross-validaion r^2 scores

print('The R^2 value for Linear Regression is         :', cvln_r2score)
print('The R^2 value for KNN Regressor is             :', cvkn_r2score)
print('The R^2 value for SVM Regressor is             :', cvsvr_r2score)
print('The R^2 value for Random Forests Regressor is  :', cvrf_r2score)
print('The R^2 value for Decision Tree Regressor is   :', cvdt_r2score)
print('The R^2 value for AdaBoost Regressor is        :', cvada_r2score)
print('The R^2 value for XGBoost Regressor is         :', cvxgb_r2score)

# printing cross-validation adjusted r^2 scores

print('The Adj. R^2 value for Linear Regression is        :', cvln_adjscore)
print('The Adj. R^2 value for KNN Regressor is            :', cvkn_adjscore)
print('The Adj. R^2 value for SVM Regressor is            :', cvsvr_adjscore)
print('The Adj. R^2 value for Random Forests Regressor is :', cvrf_adjscore)
print('The Adj. R^2 value for Decision Tree Regressor is  :', cvdt_adjscore)
print('The Adj. R^2 value for AdaBoost Regressor is       :', cvada_adjscore)
print('The Adj. R^2 value for XGBoost Regressor is        :', cvxgb_adjscore)

# printing cross-validation rmse scores

print('The RSME value for Linear Regression is         :', cvln_rmsescore)
print('The RSME value for KNN Regressor is             :', cvkn_rmsescore)
print('The RSME value for SVM Regressor is             :', cvsvr_rmsescore)
print('The RSME value for Random Forests Regressor is  :', cvrf_rmsescore)
print('The RSME value for Decision Tree Regressor is   :', cvdt_rmsescore)
print('The RSME value for AdaBoost Regressor is        :', cvada_rmsescore)
print('The RSME value for XGBoost Regressor is         :', cvxgb_rmsescore)

# printing cross-validation mae scores

print('The MAE value for Linear Regression is        :', cvln_maescore)
print('The MAE value for KNN Regressor is            :', cvkn_maescore)
print('The MAE value for SVM Regressor is            :', cvsvr_maescore)
print('The MAE value for Random Forests Regressor is :', cvrf_maescore)
print('The MAE value for Decision Tree Regressor is  :', cvdt_maescore)
print('The MAE value for AdaBoost Regressor is       :', cvada_maescore)
print('The MAE value for XGBoost Regressor is        :', cvxgb_maescore)

The R^2 value for Linear Regression is         : 0.3847
The R^2 value for KNN Regressor is             : 0.5889
The R^2 value for SVM Regressor is             : 0.4427
The R^2 value for Random Forests Regressor is  : 0.6403
The R^2 value for Decision Tree Regressor is   : 0.4771
The R^2 value for AdaBoost Regressor is        : 0.4305
The R^2 value for XGBoost Regressor is         : 0.6357
The Adj. R^2 value for Linear Regression is        : 0.3838
The Adj. R^2 value for KNN Regressor is            : 0.5883
The Adj. R^2 value for SVM Regressor is            : 0.4419
The Adj. R^2 value for Random Forests Regressor is : 0.6397
The Adj. R^2 value for Decision Tree Regressor is  : 0.4763
The Adj. R^2 value for AdaBoost Regressor is       : 0.4296
The Adj. R^2 value for XGBoost Regressor is        : 0.6351
The RSME value for Linear Regression is         : 32.8
The RSME value for KNN Regressor is             : 26.63
The RSME value for SVM Regressor is             : 43.07
The RSME value for Ra